# Horizon 1.0.0

Welcome to this humble introduction to [**Horizon**](https://github.com/FrancescoRuscelli/horizon_gui)!

Horizon is a framework for trajectory optimization and optimal control with a special focus on robotics systems.
It is particulary useful to describe and solve parametric nolinear problems for trajectory planning, both off-line and in a MPC fashion.

Horizon is based on [**CasADi**](https://web.casadi.org/), a powerful tool for nonlinear optimization and algorithmic differentiation.

---

You can find many examples in the Horizon repo and a wonderful [**documentation**](https://francescoruscelli.github.io/horizon_gui), but if what you desire is a simple, interactive, FUN introduction to this framework, keep reading!


Let's start by importing the module horizon.
In particular, we are intrested in the submodule Problem, which allows the creation of our optimization problem.
While we are at it, we're importing some other useful packages.

In [ ]:
from horizon.problem import Problem
import horizon.utils.transcription_methods as transmet
import horizon.utils.plotter as plotter
import casadi as cs
import numpy as np
import matplotlib.pyplot as plt

Great! Now let's define a number of shooting nodes.

---
## Intro

Oh wait.. I'll give you a small introduction to what this is all about.

Say you need to plan a given trajectory for your *dynamical system*. This system is described by one or many differential equation, which determines its evolution in time, depending on its *state* and *inputs*.\
Of course you don't want any trajectory, but one that satisfies some desired *constraints* and minimizes a given *performance index*. In addition to this, you may want to add some *boundaries* to your variables, cause that motor of yours cannot exert infinite torque right?\
Finally, you want your problem *parametric*: for instance, you want to parametrize your reference trajectory, so that it can be changed to suit your requirements.

In optimal control literature, there are many strategies that can be employed to find a solution. Sadly, I don't know them. Also I can't teach (especially you, you're too old).
Horizon follows the philosophy behind the direct methods: reducing your optimal control problem to a **nonlinear program (NLP)**.

To solve a NLP, there are two viable ways that are supported by the Horizon framework:

- Direct collocation
- Multiple shooting

Both these methods, before optimizing, discretize the NLP into a finite dimensional problem: in other words, the horizon of the planning is divided into a given number of nodes. Of course, this means that your optimal input will get discretized as well. If you are ok with that, let's get a little more technical.

By the way, I'm not goint to explain the internal mechanics of these direct methods, because, hey, Horizon is taking care of it for you!

---

## Problem definition

Suppose we have a (unit) mass on a surface, and we need it to move to a given point by exerting a force on it. In doing so, we want to minimize the force, and we want to get there in a given time.\
However, an obstacle is between the initial position of the mass and our goal. 

How to do?

First thing to do is discretizing. Then, we can set up the problem. 

In [ ]:
T = 5
dt = 0.1
N = int(T/dt)

prb = Problem(N)

It's time to define the variables of our system.\
As you can probably guess, the relevant quantities are divided into:
- state variables
- input variables

In [ ]:
p = prb.createStateVariable('pos', dim=2) # position of the mass in the x/y plane
v = prb.createStateVariable('vel', dim=2) # velocity of the mass
F = prb.createInputVariable('force', dim=2) # force exerted to the mass

As you can tell, you only receive an handle to one single variable.\
But we said that the system is discretized, and for each node exists a decision variable to be optimized... where are all the variable of the problem?\
Don't worry. It's burdensome to manage all of them, so horizon does it for you. If you really want to see them, try the following code:

In [ ]:
p.getImpl()

In short, each 'abstract' value you are dealing with holds its own decision variables at each node, that are going to be used by the solver.

--

The horizon problem stores the state variable and the input variable separately.\ It is worth mentioning that the state variable are defined on **all** the *N* nodes, while the input variables transition from one state to the other, hence they are defined on *N-1* nodes.

One can easily retrieve the whole state of the system, as well as the input.

In [ ]:
sys_state = prb.getState()
sys_input = prb.getInput()

Now let's inject the dynamics in the problem. After all, horizon need to know how the system evolves in time. To do so, it asks for a vector that specifies the evolution of the state variables.
In this specific scenario, the state $x$ is and its derivative $\dot{x}$ are:

\begin{equation}
x = \begin{bmatrix} p \\ v \end{bmatrix}
\qquad
\dot{x} = \begin{bmatrix} v \\ F \end{bmatrix} 
\end{equation}

If you're wondering where is the mass, it is unitary. So:

In [ ]:
xdot = cs.vertcat(v, F)
prb.setDynamics(xdot)

As mentioned below, horizon provides two methods to solve the problem, the *multiple shooting* and the *direct collocation*. I won't explain in detail how they work, you can search it up, but hey... you don't need to! Just tell horizon which method you want to use, and it will do its magic:

In [ ]:
th = transmet.TranscriptionsHandler(prb, dt)
th.setMultipleShooting()
# or, if you want to try another direct method:
#th.setDirectCollocation(3) # 3 is the order of the polynomial that approximate the dynamics

Nevermind, a brief explanation can't hurt.
Integrating the system dynamics given as initial condition the state at each node and a constant input, we get a discontinuous trajectory, as shown in the following picture.

![discontinuous trajectory](img/multishoot_1.png)

The multiple shooting method simply enforces continuity of the system trajectory between each node. In other words, the final state obtained by intregrating the system starting from the state at a node $k$ (with a constant input) must be equal to the state variable at node $k+1$.\
If the system satisfy this condition, we will get something on these lines:
![continuous trajectory](img/multishoot_2.png)

---
Let's continue. What we need now is to set some bounds. To do so, you just need to grab your variables...

In [ ]:
# set initial state (rest in zero)
p.setBounds(lb=[0, 0], ub=[0, 0], nodes=0)
v.setBounds(lb=[0, 0], ub=[0, 0], nodes=0)

# final constraint
p.setBounds(lb=[1, 1], ub=[1, 1], nodes=N)
v.setBounds(lb=[0, 0], ub=[0, 0], nodes=N)

As you can see, these lines bound position and velocity at the very first node (nodes=0) and the last node (nodes=N).

Now let's insert the obstacle. It's going to be exactly where we don't want. In between our starting position and our coveted goal.

In [ ]:
obs_center = np.array([0.5, 0.5])
obs_r = 0.4
obs = cs.sumsqr(p - obs_center) - obs_r

obs_cnsrt = prb.createIntermediateConstraint('obstacle', obs)
obs_cnsrt.setUpperBounds(np.inf)

Still following? Hang in there, we're almost done!\
For simplicity, the obstacle is a convex shape also known as circle. The mass cannot enter the obstacle, hence the lower bound is zero, while the upper bound is inf: everywhere else the mass can navigate freely.

> *Hint*: if not specified, the bounds of a constraints are *zero*

The last desire we bear in our heart is minimizing the force exerted.
To do so, we need to a cost function:

In [ ]:
prb.createIntermediateCost('cost', cs.sumsqr(F))

> *Hint*: *createIntermediateConstraint/Cost* are just shortcuts to define a constraint/cost function along the "intermediate" nodes, i.e. every node but the last one. Similarly, *createFinalConstraint/Cost* set them only at the last node

That's it! We are ready to solve the problem!

In [ ]:
prb.createProblem()
solution = prb.solveProblem()

That's a lot of outputs: it's the default solver spitting out all the relevant information. What is really important, an optimal solution was found!

--
Which solution tho? Let's plot it.

In [ ]:
hplt = plotter.PlotterHorizon(prb)
hplt.plotVariables()
hplt.plotFunctions()


plt.figure()
plt.plot(solution['pos'][0], solution['pos'][1])
plt.plot([0, 0], [0, 0], 'bo', markersize=12)
plt.plot([1, 1], [1, 1], 'g*', markersize=12)
circle = plt.Circle(obs_center, radius=obs_r, fc='r')
plt.gca().add_patch(circle)

Here we are! You can see all the relevant parameters, and, in the last plot, the trajectory of our mass!
It's amazing!

> *Hint*: the horizon plotter plots automatically all the variables and the functions.

Just to give a little bit of context, the blue dot is the starting point, while the green star is the goal. The trajectory of the unit mass is in blue, and as you can see it goes around the red obstacle without touching it.

--

What if you take out the obstacle constraint? Try it!